In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

unary_operators=[np.sin, np.cos, np.exp, np.abs, np.log, np.tan]
binary_operators=[np.add, np.subtract, np.multiply, np.divide]


operators = unary_operators + binary_operators

In [2]:
problem = np.load("data/problem_0.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 1000)
y.shape: (1000,)


In [3]:
x_train,y_train,x_val,y_val = split_dataset(x,y,0.8)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(2, 800)
(2, 200)
(800,)
(200,)


In [5]:
my_node = Node(value=np.add, left=Node(value=None, feature_index=0), right=Node(value=np.multiply, left=Node(value=0.2), right=Node(value=None, feature_index=1)))
my_inv = Individual(genome=my_node)
assign_population_fitness_train([my_inv], x_train, y_train)
print(my_inv.fitness)
assign_population_fitness_val([my_inv], x_val, y_val)
print(my_inv.fitness_val)

0.00014421615402757763
0.00013792776500560924


In [3]:
conf={
"NUM_POPULATION": 100,
"NUM_GENERATIONS": 600,
"MAX_DEPTH_INITIAL": 7,
"DEDUPE_INTERVAL": 15,
"ELITISM": False,
"ELITE_COUNT": 3,
"TOURNAMENT_SUBSET_SIZE": 3,
"TOURNAMENT_WINNER_SIZE": 20,
"XOVER": 0.8,
"MUTATION": 0.8,
"BREED_NEW": 15,
"KILL_AGE": 16,
"MAX_COMPLEXITY": 150,
"CONSTANT_FIT_INTERVAL": 30,
"CONSTANT_FIT_ITERATION": 6
}

In [4]:
def evolve(train_x,train_y,val_x,val_y, conf):
    NUM_POPULATION = conf["NUM_POPULATION"]
    NUM_GENERATIONS = conf["NUM_GENERATIONS"]
    MAX_DEPTH_INITIAL = conf["MAX_DEPTH_INITIAL"]
    DEDUPE_INTERVAL = conf["DEDUPE_INTERVAL"]
    ELITISM = conf["ELITISM"]
    ELITE_COUNT = conf["ELITE_COUNT"]
    TOURNAMENT_SUBSET_SIZE = conf["TOURNAMENT_SUBSET_SIZE"]
    TOURNAMENT_WINNER_SIZE = conf["TOURNAMENT_WINNER_SIZE"]
    XOVER = conf["XOVER"]
    MUTATION = conf["MUTATION"]
    BREED_NEW = conf["BREED_NEW"]
    KILL_AGE = conf["KILL_AGE"]
    MAX_COMPLEXITY = conf["MAX_COMPLEXITY"]
    CONSTANT_FIT_INTERVAL = conf["CONSTANT_FIT_INTERVAL"]
    CONSTANT_FIT_ITERATION = conf["CONSTANT_FIT_ITERATION"]

    print("Population is creating with a size: ", NUM_POPULATION)
    my_population = create_population(NUM_POPULATION,MAX_DEPTH_INITIAL,train_x.shape[0])
    print("Population has created.")
    print("Individual fitness values are assigning")
    assign_population_fitness_train(my_population,train_x,train_y)
    assign_population_fitness_val(my_population,val_x,val_y)
    print(f" Population size: {len(my_population)}")
    print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
    print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")

    # Sort the population by fitness (ascending), then take the top 5
    top_5_individuals = top_n_individuals(my_population, 5)

    # Print the top 5 individuals with the minimum fitness
    for i, individual in enumerate(top_5_individuals, 1):
        print(f"Top {i}: Fitness = {individual.fitness:.5f} Fitness_val = {individual.fitness_val:.5f} Genome = {individual.genome}")
    #----------------------------------------------------------------------------
    assign_population_fitness_train(my_population,train_x,train_y)
    assign_population_fitness_val(my_population,val_x,val_y)
    best_result = top_n_individuals(my_population, 1)[0]
    for generation in tqdm(range(NUM_GENERATIONS)):
        num_xover=0
        num_mut=0
        num_succesfull_mut=0
        age_population(my_population)
        

        # Kill eldest individuals
        kill_eldest(my_population, KILL_AGE)
        # print("Population size before killing constants: ", len(my_population))
        
        # print("Population size after killing constants: ", len(my_population))


        best_ind = tournament_selection(my_population,TOURNAMENT_SUBSET_SIZE,TOURNAMENT_WINNER_SIZE,ELITISM=ELITISM, elite_count=ELITE_COUNT)

        
        for _ in range(BREED_NEW): # CHECK
            if random.random() < XOVER:
                # Xover the best individual and replace worst individuals
                parent1 = my_population[best_ind[np.random.choice(len(best_ind))]]
                parent2 = my_population[best_ind[np.random.choice(len(best_ind))]]
                child1, child2 = crossover(parent1,parent2)
                my_population.append(child1)
                my_population.append(child2)
                num_xover+=1

            if random.random() < MUTATION:
                # Mutate best individuals and replace worst individuals
                parent = my_population[best_ind[np.random.choice(len(best_ind))]]
                
                child, success = mutation_w_sa(parent, x.shape[0], train_x, train_y)
                num_mut+=1
                if success:                
                    my_population.append(child)
                    num_succesfull_mut+=1

        assign_population_fitness_train(my_population,train_x,train_y)
        assign_population_fitness_val(my_population,val_x,val_y)

        # print(f"xover: {num_xover}, mut: {num_mut}, succesfull mut: {num_succesfull_mut}")
        simplify_constant_population(my_population)
        # Update elites age as 0
        elites = top_n_individuals(my_population, ELITE_COUNT)
        if best_result.fitness_val > elites[0].fitness_val:
            best_result = elites[0]
            print(f"New best result found: {best_result}")
        
        for elite in elites:
            my_population.remove(elite)
            elite.age = 0
            my_population.append(elite)

        # Deduplicate every few iterations
        if generation % DEDUPE_INTERVAL == DEDUPE_INTERVAL-1:
            init_population = len(my_population)
            my_population = deduplicate_population(my_population)
            dedup_population = len(my_population)
            simplify_operation_population(my_population)
            simplified_operations = len(my_population)
            kill_constant(my_population)
            no_constant = len(my_population)
            kill_complex(my_population, MAX_COMPLEXITY)
            no_complex = len(my_population)
            simplify_population(my_population)
            

            print(f"Initial: {init_population}, Deduplicated: {init_population-dedup_population}, Operation simplified: {dedup_population-simplified_operations}, Constant: {simplified_operations-no_constant}, Complex: {no_constant - no_complex}, Final: {no_complex}")
            print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
            print(f"Population mean fitness: {calculate_mean_fitness(my_population)}")
            print()

            elites = top_n_individuals(my_population, ELITE_COUNT)
            for elite in elites:
                my_population.remove(elite)
                elite.age = 0
                my_population.append(elite)
                
            best_5 = top_n_individuals(my_population, 5)
            for ind in range(len(best_5)):
                print(f"{ind}. Fitness: {best_5[ind].fitness:.5f}, Fitness_val = {best_5[ind].fitness_val:.5f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}, Complexity: {best_5[ind].genome.complexity}")
                
        #     # Fit constants of every individual
        # if generation % CONSTANT_FIT_INTERVAL == CONSTANT_FIT_INTERVAL-1:
        #     print(f"Generation {generation}: Mean fitness {calculate_mean_fitness(my_population)}")
        #     for ind in range(len(my_population)):
        #         stronger_inv = fit_constants(my_population[ind],CONSTANT_FIT_ITERATION, train_x, train_y)
        #         my_population[ind] = stronger_inv
        #     assign_population_fitness_val(my_population,val_x,val_y)
        #     print(f"Population mean fitness after constant fit: {calculate_mean_fitness(my_population)}")
        #     best_5 = top_n_individuals(my_population, 5)
        #     for ind in range(len(best_5)):
        #         print(f"{ind}. Fitness: {best_5[ind].fitness:.5f}, Fitness_val = {best_5[ind].fitness_val:.5f}, Genome: {best_5[ind].genome}, Age: {best_5[ind].age}, Complexity: {best_5[ind].genome.complexity}")
        

    my_population = deduplicate_population(my_population)
    #-------------------------------------------------------------------------------------------------------
    assign_population_fitness_train(my_population,train_x,train_y)
    for i in my_population:
        if i.fitness == None:
            print('Cannot assign fitness')
            return my_population
    assign_population_fitness_val(my_population,val_x,val_y)
    for i in my_population:
        if i.fitness_val == None:
            print('Cannot assign fitness_val')
            return my_population
    # Sort the population by fitness (ascending), then take the top 5
    top_5_individuals = top_n_individuals(my_population, 15)

    # Print the best individual
    print(f"Best individual: Fitness = {best_result.fitness}: Genome = {best_result.genome}")
    # Print the top 5 individuals with the minimum fitness in population
    for i, individual in enumerate(top_5_individuals, 1):
        print(f"Top {i}: Fitness = {individual.fitness} Fitness_val = {individual.fitness_val} Genome = {individual.genome}")

    # Print the mean fitness of the population
    print("Mean fitness of the population: ", calculate_mean_fitness(my_population))
    print("Population size: ", len(my_population))
    print(f"Population mean complexity: {calculate_mean_complexity(my_population)}")
    
    return my_population

In [9]:

problem = np.load("data/problem_0.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
my_pop_0 = evolve(x_train,y_train,x_val,y_val,conf)


x.shape: (2, 1000)
y.shape: (1000,)
x_train.shape:  (2, 800)
x_val.shape:  (2, 200)
y_train.shape:  (800,)
y_val.shape:  (200,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 59
Population mean fitness: 210.9997387484309
Population mean complexity: 2163.2881355932204
Top 1: Fitness = 1.87549 Fitness_val = 1.95333 Genome = sin(x[0])
Top 2: Fitness = 3.41310 Fitness_val = 3.46046 Genome = exp([-1.40693152])
Top 3: Fitness = 3.38401 Fitness_val = 3.46620 Genome = sin([0.1588025])
Top 4: Fitness = 3.46352 Fitness_val = 3.47189 Genome = ([-0.32161835] * [-1.06341941])
Top 5: Fitness = 3.54582 Fitness_val = 3.47286 Genome = (x[1] * x[1])


  0%|          | 1/600 [00:01<16:20,  1.64s/it]

New best result found: Individual(genome=x[0], fitness=np.float64(0.010763171848597946), fitness_val=np.float64(0.010325632633118373), age=0, T=1)


  2%|▎         | 15/600 [00:12<04:51,  2.01it/s]

Initial: 586, Deduplicated: 403, Operation simplified: 0, Constant: 111, Complex: 29, Final: 43
Population mean complexity: 23.837209302325583
Population mean fitness: 168.88178322812394

0. Fitness: 0.01076, Fitness_val = 0.01033, Genome: x[0], Age: 0, Complexity: 1
1. Fitness: 1.67647, Fitness_val = 1.73689, Genome: tan(sin(x[0])), Age: 0, Complexity: 36
2. Fitness: 1.87549, Fitness_val = 1.95333, Genome: sin(x[0]), Age: 0, Complexity: 8
3. Fitness: 1.97661, Fitness_val = 2.06181, Genome: sin(sin(x[0])), Age: 6, Complexity: 36
4. Fitness: 3.43332, Fitness_val = 3.24075, Genome: exp(sin(x[0])), Age: 8, Complexity: 45


...

Initial: 570, Deduplicated: 205, Operation simplified: 0, Constant: 5, Complex: 76, Final: 284
Population mean complexity: 78.8169014084507
Population mean fitness: 2077.716986352439

0. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + (((x[0] + (x[1] - sin(sin(x[1])))) + (x[1] - x[1])) - x[1])) + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])), Age: 0, Complexity: 105
1. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + ((((sin(x[1]) + (x[0] - sin(x[1]))) + (sin(x[1]) - sin(sin(x[1])))) + (x[1] - sin(x[1]))) - x[1])) + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])), Age: 2, Complexity: 137
2. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + (x[0] - x[1])) + (sin(x[1]) - sin(sin(x[1])))) + (x[1] - sin(sin(x[1])))), Age: 0, Complexity: 97
3. Fitness: 0.00182, Fitness_val = 0.00188, Genome: ((((sin(x[1]) - sin(x[1])) + (x[0] - x[1])) + (x[1] - sin((sin(x[1]) - x[1])))) + (sin(x[1]) - sin(sin(x[1])))), Age: 1, Complexity: 114
4. Fitness: 0.00183,

 98%|█████████▊| 585/600 [09:00<00:27,  1.81s/it]

Initial: 526, Deduplicated: 168, Operation simplified: 0, Constant: 8, Complex: 86, Final: 264
Population mean complexity: 80.70075757575758
Population mean fitness: 20910233983343.523

0. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + (((x[0] + (x[1] - sin(sin(x[1])))) + (x[1] - x[1])) - x[1])) + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])), Age: 0, Complexity: 105
1. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + ((((sin(x[1]) + (x[0] - sin(x[1]))) + (sin(x[1]) - sin(sin(x[1])))) + (x[1] - sin(x[1]))) - x[1])) + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])), Age: 9, Complexity: 137
2. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + (x[0] - x[1])) + (sin(x[1]) - sin(sin(x[1])))) + (x[1] - sin(sin(x[1])))), Age: 0, Complexity: 97
3. Fitness: 0.00182, Fitness_val = 0.00188, Genome: ((((sin(x[1]) - sin(x[1])) + (x[0] - x[1])) + (x[1] - sin((sin(x[1]) - x[1])))) + (sin(x[1]) - sin(sin(x[1])))), Age: 16, Complexity: 114
4. Fitness: 0.001

100%|██████████| 600/600 [09:25<00:00,  1.06it/s]

Initial: 559, Deduplicated: 208, Operation simplified: 0, Constant: 1, Complex: 96, Final: 254
Population mean complexity: 81.32283464566929
Population mean fitness: 13742002.029475426

0. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + (((x[0] + (x[1] - sin(sin(x[1])))) + (x[1] - x[1])) - x[1])) + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])), Age: 0, Complexity: 105
1. Fitness: 0.00166, Fitness_val = 0.00170, Genome: ((sin(x[1]) + (((x[0] + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])) - x[1])) + (x[1] - sin(sin(x[1])))), Age: 0, Complexity: 101
2. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + ((((sin(x[1]) + (x[0] - sin(x[1]))) + (sin(x[1]) - sin(sin(x[1])))) + (x[1] - sin(x[1]))) - x[1])) + (x[1] - sin(sin(x[1])))) + (sin(x[1]) - x[1])), Age: 0, Complexity: 137
3. Fitness: 0.00166, Fitness_val = 0.00170, Genome: (((sin(x[1]) + (x[0] - x[1])) + (sin(x[1]) - sin(sin(x[1])))) + (x[1] - sin(sin(x[1])))), Age: 7, Complexity: 97
4. Fitness: 0.00183, Fi

In [10]:

problem = np.load("data/problem_1.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
my_pop_1 = evolve(x_train,y_train,x_val,y_val,conf)


x.shape: (1, 500)
y.shape: (500,)
x_train.shape:  (2, 800)
x_val.shape:  (2, 200)
y_train.shape:  (800,)
y_val.shape:  (200,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 56
Population mean fitness: 44.60898124372013
Population mean complexity: 966.9285714285714
Top 1: Fitness = 0.00000 Fitness_val = 0.00000 Genome = sin(x[0])
Top 2: Fitness = 0.05476 Fitness_val = 0.05025 Genome = tan(x[0])
Top 3: Fitness = 0.25499 Fitness_val = 0.25180 Genome = (x[0] - x[0])
Top 4: Fitness = 0.25738 Fitness_val = 0.25578 Genome = ([0.04965304] * [0.6201221])
Top 5: Fitness = 0.26006 Fitness_val = 0.25727 Genome = (x[0] * [-0.12596665])


  2%|▎         | 15/600 [00:06<03:29,  2.79it/s]

Initial: 513, Deduplicated: 340, Operation simplified: 0, Constant: 114, Complex: 11, Final: 48
Population mean complexity: 13.041666666666666
Population mean fitness: 20.061174747056697

0. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(x[0]), Age: 0, Complexity: 8
1. Fitness: 0.00156, Fitness_val = 0.00148, Genome: sin(sin(x[0])), Age: 0, Complexity: 36
2. Fitness: 0.00333, Fitness_val = 0.00310, Genome: x[0], Age: 0, Complexity: 1
3. Fitness: 0.00481, Fitness_val = 0.00457, Genome: sin(sin(sin(x[0]))), Age: 6, Complexity: 148
4. Fitness: 0.05476, Fitness_val = 0.05025, Genome: tan(x[0]), Age: 3, Complexity: 8


  5%|▌         | 30/600 [00:13<03:27,  2.75it/s]

Initial: 461, Deduplicated: 324, Operation simplified: 0, Constant: 73, Complex: 6, Final: 58
Population mean complexity: 24.03448275862069
Population mean fitness: 121.30601487264863

0. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(x[0]), Age: 0, Complexity: 8
1. Fitness: 0.00125, Fitness_val = 0.00119, Genome: tan(sin(sin(x[0]))), Age: 0, Complexity: 148
2. Fitness: 0.00156, Fitness_val = 0.00148, Genome: sin(sin(x[0])), Age: 0, Complexity: 36
3. Fitness: 0.00333, Fitness_val = 0.00310, Genome: x[0], Age: 0, Complexity: 1
4. Fitness: 0.00481, Fitness_val = 0.00457, Genome: sin(sin(sin(x[0]))), Age: 11, Complexity: 148


...

Initial: 471, Deduplicated: 317, Operation simplified: 0, Constant: 73, Complex: 23, Final: 58
Population mean complexity: 41.08620689655172
Population mean fitness: 0.4814856355710402

0. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(x[0]), Age: 0, Complexity: 8
1. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(((x[0] * x[0]) / x[0])), Age: 0, Complexity: 24
2. Fitness: 0.00044, Fitness_val = 0.00044, Genome: sin(sin(tan(x[0]))), Age: 0, Complexity: 148
3. Fitness: 0.00089, Fitness_val = 0.00086, Genome: sin(tan(sin(x[0]))), Age: 12, Complexity: 148
4. Fitness: 0.00125, Fitness_val = 0.00119, Genome: tan(sin(sin(x[0]))), Age: 7, Complexity: 148


 98%|█████████▊| 585/600 [04:00<00:05,  2.78it/s]

Initial: 459, Deduplicated: 310, Operation simplified: 0, Constant: 86, Complex: 6, Final: 57
Population mean complexity: 37.63157894736842
Population mean fitness: 11720.46430142706

0. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(x[0]), Age: 0, Complexity: 8
1. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(((x[0] * x[0]) / x[0])), Age: 0, Complexity: 24
2. Fitness: 0.00044, Fitness_val = 0.00044, Genome: sin(sin(tan(x[0]))), Age: 0, Complexity: 148
3. Fitness: 0.00089, Fitness_val = 0.00086, Genome: sin(tan(sin(x[0]))), Age: 1, Complexity: 148
4. Fitness: 0.00125, Fitness_val = 0.00119, Genome: tan(sin(sin(x[0]))), Age: 5, Complexity: 148


100%|██████████| 600/600 [04:05<00:00,  2.44it/s]

Initial: 467, Deduplicated: 322, Operation simplified: 0, Constant: 56, Complex: 21, Final: 68
Population mean complexity: 44.279411764705884
Population mean fitness: 9825.4075983807

0. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(x[0]), Age: 0, Complexity: 8
1. Fitness: 0.00000, Fitness_val = 0.00000, Genome: sin(((x[0] * x[0]) / x[0])), Age: 0, Complexity: 24
2. Fitness: 0.00000, Fitness_val = 0.00000, Genome: ((x[0] * sin(x[0])) / x[0]), Age: 0, Complexity: 12
3. Fitness: 0.00044, Fitness_val = 0.00044, Genome: sin(sin(tan(x[0]))), Age: 11, Complexity: 148
4. Fitness: 0.00089, Fitness_val = 0.00086, Genome: sin(tan(sin(x[0]))), Age: 7, Complexity: 148
Best individual: Fitness = 7.0537184213182515e-34: Genome = sin(x[0])
Top 1: Fitness = 7.0537184213182515e-34 Fitness_val = 7.414830285890858e-34 Genome = sin(x[0])
Top 2: Fitness = 1.083847168859298e-33 Fitness_val = 1.020742870525235e-33 Genome = sin(((x[0] * x[0]) / x[0]))
Top 3: Fitness = 1.1555880592877225e-33 Fitness_val

In [11]:
problem = np.load("data/problem_2.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
my_pop_2 = evolve(x_train,y_train,x_val,y_val,conf)

x.shape: (3, 5000)
y.shape: (5000,)
x_train.shape:  (1, 400)
x_val.shape:  (1, 100)
y_train.shape:  (400,)
y_val.shape:  (100,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 55
Population mean fitness: 29524809706919.81
Population mean complexity: 983.9090909090909
Top 1: Fitness = 29524701472459.54297 Fitness_val = 29985551587121.13672 Genome = exp(x[0])
Top 2: Fitness = 29524780499453.81641 Fitness_val = 29985634520632.17578 Genome = exp(x[2])
Top 3: Fitness = 29524803649596.75781 Fitness_val = 29985665929217.57031 Genome = (x[1] + x[1])
Top 4: Fitness = 29524804102304.65625 Fitness_val = 29985667214708.00000 Genome = (x[2] + x[1])
Top 5: Fitness = 29524810638326.23047 Fitness_val = 29985671172007.06250 Genome = (x[0] * x[0])


  0%|          | 2/600 [00:24<1:55:58, 11.64s/it]

New best result found: Individual(genome=(exp(x[0]) + x[1]), fitness=np.float64(29524696486520.35), fitness_val=np.float64(29985545837800.336), age=0, T=1)


  0%|          | 3/600 [00:30<1:31:10,  9.16s/it]

New best result found: Individual(genome=(exp(x[2]) * x[0]), fitness=np.float64(29524593228589.055), fitness_val=np.float64(29985492699982.316), age=0, T=1)


...

0. Fitness: 24701781006323.71875, Fitness_val = 25712405860069.13672, Genome: (((((x[0] - ((x[0] * x[1]) * x[2])) - (x[0] * (x[0] * x[1]))) + ((x[0] + (((((x[0] - ((x[0] * x[1]) * x[2])) + (((((((x[0] - (((x[0] - x[2]) * x[0]) * x[2])) + x[0]) + (x[0] + x[0])) + x[0]) - x[1]) + x[0]) + x[0])) - x[1]) - x[1]) + x[0])) + abs(x[0]))) * ((x[0] + ((x[0] + ((x[0] - ((x[0] * x[1]) * x[0])) + (x[0] + ((x[0] - ((x[0] * x[1]) * x[2])) + x[0])))) + x[0])) + (x[0] + (((x[0] + (((x[2] + ((x[0] - (((x[0] - x[2]) * x[0]) * x[2])) + x[0])) + x[0]) + x[0])) + x[0]) + x[0])))) * ((((x[0] - (x[0] * x[2])) * x[0]) - (x[0] * (x[0] * x[1]))) + (((((x[0] - x[1]) + x[0]) + ((x[0] - ((x[0] * x[1]) * x[2])) + (x[0] + (x[0] + x[0])))) + (x[0] + x[0])) + x[0]))), Age: 0, Complexity: 148
1. Fitness: 24836403220047.37500, Fitness_val = 25745647672958.53906, Genome: (((((x[0] - ((x[0] * x[1]) * x[2])) - (x[0] * (x[0] * x[1]))) + ((x[0] + (((((x[0] - ((x[0] * x[1]) * x[2])) + (((((((x[0] - (((x[0] - x[2]) * x[0]) * x

100%|█████████▉| 599/600 [4:26:57<00:41, 41.80s/it]

Initial: 568, Deduplicated: 34, Operation simplified: 0, Constant: 0, Complex: 189, Final: 345
Population mean complexity: 137.73333333333332
Population mean fitness: 39574491022102.96



100%|██████████| 600/600 [4:27:37<00:00, 26.76s/it]

0. Fitness: 24701781006323.71875, Fitness_val = 25712405860069.13672, Genome: (((((x[0] - ((x[0] * x[1]) * x[2])) - (x[0] * (x[0] * x[1]))) + ((x[0] + (((((x[0] - ((x[0] * x[1]) * x[2])) + (((((((x[0] - (((x[0] - x[2]) * x[0]) * x[2])) + x[0]) + (x[0] + x[0])) + x[0]) - x[1]) + x[0]) + x[0])) - x[1]) - x[1]) + x[0])) + abs(x[0]))) * ((x[0] + ((x[0] + ((x[0] - ((x[0] * x[1]) * x[0])) + (x[0] + ((x[0] - ((x[0] * x[1]) * x[2])) + x[0])))) + x[0])) + (x[0] + (((x[0] + (((x[2] + ((x[0] - (((x[0] - x[2]) * x[0]) * x[2])) + x[0])) + x[0]) + x[0])) + x[0]) + x[0])))) * ((((x[0] - (x[0] * x[2])) * x[0]) - (x[0] * (x[0] * x[1]))) + (((((x[0] - x[1]) + x[0]) + ((x[0] - ((x[0] * x[1]) * x[2])) + (x[0] + (x[0] + x[0])))) + (x[0] + x[0])) + x[0]))), Age: 0, Complexity: 148
1. Fitness: 24836403220047.37500, Fitness_val = 25745647672958.53906, Genome: (((((x[0] - ((x[0] * x[1]) * x[2])) - (x[0] * (x[0] * x[1]))) + ((x[0] + (((((x[0] - ((x[0] * x[1]) * x[2])) + (((((((x[0] - (((x[0] - x[2]) * x[0]) * x

In [4]:
conf["MAX_DEPTH_INITIAL"] = 7
conf["MAX_COMPLEXITY"] = 150

In [6]:
problem = np.load("data/problem_3.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
x_train, y_train, x_val, y_val = split_dataset(x[:,:1000],y[:1000],0.8)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
my_pop_3 = evolve(x_train,y_train,x_val,y_val,conf)

x.shape: (3, 5000)
y.shape: (5000,)
x_train.shape:  (3, 800)
x_val.shape:  (3, 200)
y_train.shape:  (800,)
y_val.shape:  (200,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 55
Population mean fitness: 2881.474525219636
Population mean complexity: 1065.8363636363636
Top 1: Fitness = 2716.53769 Fitness_val = 2553.09164 Genome = exp(x[0])
Top 2: Fitness = 2667.37719 Fitness_val = 2746.52960 Genome = tan([1.36412633])
Top 3: Fitness = 2710.64799 Fitness_val = 2809.69084 Genome = exp([1.23372264])
Top 4: Fitness = 2720.26248 Fitness_val = 2823.51462 Genome = ([-1.90807936] / [-0.60547923])
Top 5: Fitness = 2742.00725 Fitness_val = 2865.48502 Genome = abs(x[2])


  0%|          | 1/600 [00:03<38:07,  3.82s/it]

New best result found: Individual(genome=(x[1] * x[1]), fitness=np.float64(2613.550126004394), fitness_val=np.float64(2505.6195112219652), age=0, T=1)


  1%|          | 4/600 [00:14<36:31,  3.68s/it]

New best result found: Individual(genome=exp([3.15135395]), fitness=np.float64(2435.543789019943), fitness_val=np.float64(2237.713848311181), age=0, T=1)


...

  2%|▏         | 14/600 [01:00<49:52,  5.11s/it]

New best result found: Individual(genome=(exp([3.15135395]) + tan(([4.51548028] * cos((sin(abs(cos((x[0] / x[1])))) * cos((tan([-4.85752494]) + [4.51548028]))))))), fitness=np.float64(2467.0262890605927), fitness_val=np.float64(2220.2685359013935), age=0, T=1)


  2%|▏         | 14/600 [01:06<46:04,  4.72s/it]

Simplified genome: (x[0] + x[2])
Original genome: (x[0] + x[2])
Simplified genome: ([0.65289924] + x[2])
Original genome: ([0.65289924] + x[2])
Simplified genome: ([0.72625855] + x[2])
Original genome: (x[2] + [0.72625855])
Simplified genome: ((-1 * x[0]) + x[1])
Original genome: (x[1] - x[0])
Simplified genome: cos(x[0])
Original genome: cos(x[0])
Simplified genome: ([-1.24453998] * x[2])
Original genome: ([-1.24453998] * x[2])
Simplified genome: ([-1.42102378] + x[1])
Original genome: (x[1] - [1.42102378])
Simplified genome: ([0.70692579] * x[1])
Original genome: (x[1] * [0.70692579])
Simplified genome: tan(x[2])
Original genome: tan(x[2])
Simplified genome: ([-0.746572] + (-1 * x[2]))
Original genome: ([-0.746572] - x[2])
Simplified genome: ([0.58917039] + (-1 * x[0]))
Original genome: ([0.58917039] - x[0])
Simplified genome: ([0.43987855] * x[2])
Original genome: (x[2] * [0.43987855])
Simplified genome: ([-0.13273082] + (-1 * x[0]))
Original genome: ([-0.13273082] - x[0])
No simpli

StopIteration: 

In [13]:
problem = np.load("data/problem_4.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
my_pop_4 = evolve(x_train,y_train,x_val,y_val,conf)


x.shape: (2, 5000)
y.shape: (5000,)
x_train.shape:  (2, 4000)
x_val.shape:  (2, 1000)
y_train.shape:  (4000,)
y_val.shape:  (1000,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 50
Population mean fitness: 424.182134192444
Population mean complexity: 234.76
Top 1: Fitness = 21.54983 Fitness_val = 22.08658 Genome = exp([0.59455368])
Top 2: Fitness = 21.74200 Fitness_val = 22.38322 Genome = exp([0.40649865])
Top 3: Fitness = 22.57378 Fitness_val = 23.29537 Genome = (sin((sin(abs(((x[0] - [0.44041989]) + abs([-0.12643391])))) - (sin(sin(([0.69438677] * [-1.03459123]))) - ((exp([-1.42742308]) - tan([0.33236254])) / (tan(x[0]) - (x[0] - x[1])))))) + tan(sin(sin(tan(abs(cos(x[0])))))))
Top 4: Fitness = 22.69284 Fitness_val = 23.54105 Genome = sin([2.05290583])
Top 5: Fitness = 22.75268 Fitness_val = 23.61010 Genome = tan([0.70950123])


  0%|          | 2/600 [00:26<2:13:54, 13.44s/it]

New best result found: Individual(genome=[1.8200091], fitness=np.float64(21.547495781629152), fitness_val=np.float64(22.081623812030095), age=0, T=1)


  0%|          | 3/600 [00:36<1:59:43, 12.03s/it]

New best result found: Individual(genome=[2.17851859], fitness=np.float64(21.57123331758726), fitness_val=np.float64(21.98480123179138), age=0, T=1)


...

Initial: 555, Deduplicated: 99, Operation simplified: 0, Constant: 3, Complex: 144, Final: 309
Population mean complexity: 110.26860841423948
Population mean fitness: 362.1651566601867

0. Fitness: 0.13539, Fitness_val = 0.13873, Genome: ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(x[1])))))) + (exp(cos(x[1])) + exp(cos(x[1])))), Age: 1, Complexity: 146
1. Fitness: 0.14917, Fitness_val = 0.14880, Genome: ((abs(sin(x[1])) + ((cos(x[1]) + cos(x[1])) + ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(x[1])))))) + abs((cos(x[1]) + cos(x[1])))))) + abs(sin((x[1] + cos(x[1]))))), Age: 0, Complexity: 147
2. Fitness: 0.24570, Fitness_val = 0.22375, Genome: ((cos(x[1]) + ((cos(x[1]) + cos(x[1])) + ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(sin(x[0]))))))) + abs((cos(x[1]) + cos(x[1])))))) + abs(sin(x[0]))), Age: 0, Complexity: 138
3. Fitness: 0.24389, Fitness_val = 0.23364, Genome: ((abs(sin(x[1])) + ((cos(x

 98%|█████████▊| 585/600 [1:29:45<02:36, 10.45s/it]

Initial: 558, Deduplicated: 124, Operation simplified: 0, Constant: 3, Complex: 141, Final: 290
Population mean complexity: 107.30344827586207
Population mean fitness: 99.64921978447452

0. Fitness: 0.13539, Fitness_val = 0.13873, Genome: ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(x[1])))))) + (exp(cos(x[1])) + exp(cos(x[1])))), Age: 0, Complexity: 146
1. Fitness: 0.14917, Fitness_val = 0.14880, Genome: ((abs(sin(x[1])) + ((cos(x[1]) + cos(x[1])) + ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(x[1])))))) + abs((cos(x[1]) + cos(x[1])))))) + abs(sin((x[1] + cos(x[1]))))), Age: 0, Complexity: 147
2. Fitness: 0.24570, Fitness_val = 0.22375, Genome: ((cos(x[1]) + ((cos(x[1]) + cos(x[1])) + ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(sin(x[0]))))))) + abs((cos(x[1]) + cos(x[1])))))) + abs(sin(x[0]))), Age: 0, Complexity: 138
3. Fitness: 0.24389, Fitness_val = 0.23364, Genome: ((abs(sin(x[1])) + ((cos(

100%|██████████| 600/600 [1:32:33<00:00,  9.26s/it]

Initial: 539, Deduplicated: 103, Operation simplified: 0, Constant: 2, Complex: 149, Final: 285
Population mean complexity: 112.01754385964912
Population mean fitness: 244.600305552849

0. Fitness: 0.13539, Fitness_val = 0.13873, Genome: ((abs(sin(x[1])) + ((cos(x[1]) + cos(x[1])) + (cos(x[1]) + (cos(x[1]) + cos(x[1]))))) + (exp(cos(x[1])) + exp(cos(x[1])))), Age: 0, Complexity: 146
1. Fitness: 0.14917, Fitness_val = 0.14880, Genome: ((abs(sin(x[1])) + ((cos(x[1]) + cos(x[1])) + ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(x[1])))))) + abs((cos(x[1]) + cos(x[1])))))) + abs(sin((x[1] + cos(x[1]))))), Age: 0, Complexity: 147
2. Fitness: 0.23200, Fitness_val = 0.22222, Genome: ((abs(sin(x[1])) + ((cos(x[1]) + cos(x[1])) + ((abs(sin(x[1])) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + (cos(x[1]) + cos(x[1])))))) + abs((cos(x[1]) + cos(x[1])))))) + abs(sin(sin(x[0])))), Age: 0, Complexity: 139
3. Fitness: 0.24570, Fitness_val = 0.22375, Genome: ((cos(x[1]) + ((cos(x

In [14]:

problem = np.load("data/problem_5.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
my_pop_5 = evolve(x_train,y_train,x_val,y_val,conf)


x.shape: (2, 5000)
y.shape: (5000,)
x_train.shape:  (2, 4000)
x_val.shape:  (2, 1000)
y_train.shape:  (4000,)
y_val.shape:  (1000,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 64
Population mean fitness: 4.926271615528446e+123
Population mean complexity: 1766.46875
Top 1: Fitness = 0.00000 Fitness_val = 0.00000 Genome = log(cos(abs(log(cos(tan(log([0.75773852])))))))
Top 2: Fitness = 0.00002 Fitness_val = 0.00002 Genome = sin(abs((tan(((sin([-1.08699759]) + abs([-0.86672997])) * tan(abs([-0.3447128])))) * sin(sin(((x[1] + x[0]) - (x[1] * x[1])))))))
Top 3: Fitness = 0.06101 Fitness_val = 0.06101 Genome = ([-0.95724946] * [0.25803281])
Top 4: Fitness = 0.13273 Fitness_val = 0.13273 Genome = tan([-0.3493725])
Top 5: Fitness = 0.14105 Fitness_val = 0.14105 Genome = abs([0.3755623])


  1%|          | 5/600 [01:41<3:27:21, 20.91s/it]

New best result found: Individual(genome=log(cos(abs(log(cos(log([0.75773852])))))), fitness=np.float64(5.777044323241739e-07), fitness_val=np.float64(5.777043838510696e-07), age=0, T=1)


  1%|▏         | 8/600 [02:48<3:38:39, 22.16s/it]

New best result found: Individual(genome=sin(abs((tan(((sin([-1.08699759]) + abs([-0.86672997])) * tan(tan(((sin([-1.08699759]) + abs([-0.86672997])) * tan(abs([-0.3447128]))))))) * sin(sin(((x[1] + [0.3755623]) - (x[1] * x[1]))))))), fitness=np.float64(5.0004084798982915e-09), fitness_val=np.float64(4.96576647617728e-09), age=0, T=1)


...

New best result found: Individual(genome=([-0.00664406] * ((((x[0] * [-0.00664406]) + [1.4532548]) + [1.4532548]) * [3.20760701e-08])), fitness=np.float64(5.178762734134488e-18), fitness_val=np.float64(5.476621815195131e-18), age=0, T=1)


 18%|█▊        | 105/600 [1:16:51<9:53:09, 71.90s/it]

Initial: 427, Deduplicated: 74, Operation simplified: 0, Constant: 89, Complex: 56, Final: 208
Population mean complexity: 50.08173076923077
Population mean fitness: 8.799166981727854e+16

0. Fitness: 0.00000, Fitness_val = 0.00000, Genome: ([-0.00664406] * ((((x[0] * [-0.00664406]) + [1.4532548]) + [1.4532548]) * [3.20760701e-08])), Age: 0, Complexity: 11
1. Fitness: 0.00000, Fitness_val = 0.00000, Genome: ([-0.00664406] * ((((x[0] * ([-0.00664406] * abs(sin(((((((x[0] * [-1.04698195e-08]) + [1.4532548]) + [1.4532548]) * [3.20760701e-08]) * [-1.04698195e-08]) + [1.4532548]))))) + [1.4532548]) + [1.4532548]) * [3.20760701e-08])), Age: 0, Complexity: 69
2. Fitness: 0.00000, Fitness_val = 0.00000, Genome: ([-0.00664406] * ((((x[0] * ([-0.00664406] * abs(sin(((x[0] * [-1.04698195e-08]) + [1.4532548]))))) + [1.4532548]) + [1.4532548]) * [3.20760701e-08])), Age: 0, Complexity: 37
3. Fitness: 0.00000, Fitness_val = 0.00000, Genome: ([-0.00664406] * ((((x[0] * [-1.04698195e-08]) + [1.4532548]

 18%|█▊        | 107/600 [1:19:28<6:06:12, 44.57s/it]


KeyboardInterrupt: 

In [15]:

problem = np.load("data/problem_6.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
my_pop_6 = evolve(x_train,y_train,x_val,y_val,conf)

x.shape: (2, 5000)
y.shape: (5000,)
x_train.shape:  (2, 4000)
x_val.shape:  (2, 1000)
y_train.shape:  (4000,)
y_val.shape:  (1000,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 65
Population mean fitness: 8001.500253521459
Population mean complexity: 1552.876923076923
Top 1: Fitness = 2.68648 Fitness_val = 2.67806 Genome = (x[1] + x[1])
Top 2: Fitness = 16.23226 Fitness_val = 15.69467 Genome = ([-0.91500031] - [1.11067302])
Top 3: Fitness = 18.10004 Fitness_val = 17.46128 Genome = log(tan(tan(exp(tan(sin(tan([-1.44314053])))))))
Top 4: Fitness = 19.20174 Fitness_val = 18.51342 Genome = ([-0.649137] + [-0.62291889])
Top 5: Fitness = 20.57079 Fitness_val = 19.82736 Genome = log([0.36914677])


  2%|▎         | 15/600 [03:17<1:23:11,  8.53s/it]

Initial: 577, Deduplicated: 299, Operation simplified: 0, Constant: 139, Complex: 31, Final: 108
Population mean complexity: 25.296296296296298
Population mean fitness: 278.6312271615491

0. Fitness: 2.68648, Fitness_val = 2.67806, Genome: (x[1] + x[1]), Age: 0, Complexity: 3
1. Fitness: 8.29871, Fitness_val = 8.14984, Genome: x[1], Age: 0, Complexity: 1
2. Fitness: 9.95411, Fitness_val = 9.43270, Genome: ((x[1] + x[1]) + x[1]), Age: 0, Complexity: 5
3. Fitness: 13.49491, Fitness_val = 13.28660, Genome: (x[1] + (x[0] / x[0])), Age: 7, Complexity: 5
4. Fitness: 16.34981, Fitness_val = 16.43638, Genome: ((x[1] + x[1]) + x[0]), Age: 1, Complexity: 5


  5%|▌         | 30/600 [05:37<1:02:56,  6.63s/it]

Initial: 587, Deduplicated: 395, Operation simplified: 0, Constant: 70, Complex: 0, Final: 122
Population mean complexity: 11.721311475409836
Population mean fitness: 133.65963641788306

0. Fitness: 2.68648, Fitness_val = 2.67806, Genome: (x[1] + x[1]), Age: 0, Complexity: 3
1. Fitness: 7.65786, Fitness_val = 7.56981, Genome: (x[1] + cos(x[1])), Age: 0, Complexity: 10
2. Fitness: 8.29871, Fitness_val = 8.14984, Genome: x[1], Age: 0, Complexity: 1
3. Fitness: 8.76527, Fitness_val = 8.66912, Genome: (sin(exp((x[1] + x[1]))) + x[1]), Age: 11, Complexity: 86
4. Fitness: 9.20918, Fitness_val = 9.06654, Genome: (sin(exp(x[1])) + x[1]), Age: 12, Complexity: 46


...

New best result found: Individual(genome=(((x[1] + x[1]) + x[1]) - ((((x[0] + x[1]) - (x[0] + (x[1] / (x[1] + (((((x[0] + x[1]) - x[0]) / x[0]) - x[1]) / x[0]))))) + x[0]) + sin((((((x[0] + (x[1] / (x[1] + (((x[1] / (x[0] + (x[1] / (x[1] + (((((x[0] + x[1]) - x[0]) / x[0]) - x[1]) / x[0]))))) - x[1]) / x[0])))) + x[1]) - ((x[0] - (x[0] * (x[0] / x[0]))) + (x[1] / (x[1] + (x[1] / x[0]))))) / x[0]) - (x[0] / x[0]))))), fitness=np.float64(0.43123806973643125), fitness_val=np.float64(0.42533594208223396), age=0, T=1)


100%|█████████▉| 598/600 [2:10:56<00:54, 27.24s/it]

New best result found: Individual(genome=(((x[1] + x[1]) + x[1]) - ((((x[0] + x[1]) - (x[0] + (x[1] / (x[1] + (((((x[0] + x[1]) - x[0]) / x[0]) - x[1]) / x[0]))))) + x[0]) + sin((((((x[0] + (x[1] / (x[1] + (((x[1] / (x[0] + ((x[1] / (x[0] + (x[1] / (x[1] + (((((x[1] + x[1]) + x[1]) - (((((x[1] + x[1]) + x[1]) - (x[0] + (x[1] / (x[1] + (((((x[0] + x[1]) - x[0]) / x[0]) - x[1]) / x[0]))))) + x[0]) + sin(((((x[0] + x[1]) - ((x[0] - (x[0] + (x[1] / x[0]))) + (x[1] / (x[1] + (x[1] / x[0]))))) / x[0]) - (x[0] / x[0]))))) - x[1]) / x[0]))))) - x[1]))) - x[1]) / x[0])))) + x[1]) - ((x[0] - (x[0] + (x[0] / x[0]))) + (x[1] / (x[1] + (x[1] / x[0]))))) / x[0]) - (x[0] / x[0]))))), fitness=np.float64(0.4855479690089733), fitness_val=np.float64(0.40916292807399507), age=0, T=1)


100%|██████████| 600/600 [2:11:55<00:00, 13.19s/it]

Initial: 600, Deduplicated: 74, Operation simplified: 0, Constant: 2, Complex: 120, Final: 404
Population mean complexity: 76.86386138613861
Population mean fitness: 73124241858.52788

0. Fitness: 0.50045, Fitness_val = 0.45082, Genome: (((x[1] + x[1]) + x[1]) - ((((x[0] + x[1]) - (x[0] + (x[1] / (x[1] + (((((x[0] + x[1]) - x[0]) / x[0]) - x[1]) / x[0]))))) + x[0]) + sin(((((x[0] + x[1]) - ((x[0] - (x[0] + (x[0] / x[0]))) + (x[1] / (x[1] + (x[1] / x[0]))))) / x[0]) - (x[0] / x[0]))))), Age: 0, Complexity: 134
1. Fitness: 0.51177, Fitness_val = 0.49652, Genome: (((x[1] + x[1]) + x[1]) - (((x[1] - (x[0] + (x[1] / (x[1] + ((((x[1] + (((((x[0] + (x[1] + x[1])) - x[0]) / x[0]) - x[1]) / (x[0] + (x[1] / (x[1] + (((x[1] / (x[0] + (x[1] / x[1]))) - x[1]) / x[0])))))) / x[0]) - x[1]) / (x[0] + (x[0] / x[0]))))))) + x[0]) + x[0])), Age: 0, Complexity: 59
2. Fitness: 0.51225, Fitness_val = 0.49782, Genome: (((x[1] + x[1]) + x[1]) - (((x[1] - (x[0] + (x[1] / (x[1] + ((((x[1] + (((((x[0] + (x[1] + 

In [16]:
problem = np.load("data/problem_7.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
my_pop_7 = evolve(x_train,y_train,x_val,y_val,conf)


x.shape: (2, 5000)
y.shape: (5000,)
x_train.shape:  (2, 4000)
x_val.shape:  (2, 1000)
y_train.shape:  (4000,)
y_val.shape:  (1000,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning
 Population size: 56
Population mean fitness: 2141.743597218953
Population mean complexity: 1031.142857142857
Top 1: Fitness = 782.04315 Fitness_val = 754.54078 Genome = exp(x[0])
Top 2: Fitness = 779.68248 Fitness_val = 757.00905 Genome = tan(tan(abs(tan(abs(cos(abs([0.78516356])))))))
Top 3: Fitness = 786.32121 Fitness_val = 763.03167 Genome = tan([1.07150504])
Top 4: Fitness = 791.22784 Fitness_val = 767.26238 Genome = exp(sin(sin(abs((sin(([-0.469954] / x[0])) / sin(sin(x[0])))))))
Top 5: Fitness = 795.06895 Fitness_val = 770.75395 Genome = abs(x[0])


  0%|          | 1/600 [00:15<2:33:06, 15.34s/it]

New best result found: Individual(genome=exp(x[1]), fitness=np.float64(782.4370224225), fitness_val=np.float64(753.8593800888364), age=0, T=1)


  0%|          | 2/600 [00:28<2:22:48, 14.33s/it]

New best result found: Individual(genome=exp([0.99685928]), fitness=np.float64(772.1027945372973), fitness_val=np.float64(750.1717393026648), age=0, T=1)


...

Initial: 489, Deduplicated: 60, Operation simplified: 0, Constant: 65, Complex: 0, Final: 364
Population mean complexity: 26.557692307692307
Population mean fitness: 729.1390715600642

0. Fitness: 713.79290, Fitness_val = 702.55540, Genome: ((([0.04945817] * (([1.36981057] + abs(x[1])) + [0.18322183])) - (([0.04945817] * (([0.04945817] * x[0]) - [-0.96954168])) - [-0.96954168])) + [10.49768355]), Age: 0, Complexity: 20
1. Fitness: 713.83646, Fitness_val = 702.55557, Genome: ((([0.04945817] * (abs(([0.04945817] * ([1.4561566] + abs(x[1])))) - [10.49768355])) - [0.40570983]) + [10.49768355]), Age: 0, Complexity: 15
2. Fitness: 713.78369, Fitness_val = 702.55940, Genome: (((([0.04945817] * ([1.36981057] + abs(([0.04945817] * ([1.36981057] + abs(([0.04945817] * x[1]))))))) * ([-0.88752803] - (([1.36981057] + abs(x[1])) + [0.18322183]))) - ([0.04945817] * (([1.36981057] + abs(x[1])) + [10.49768355]))) + [10.49768355]), Age: 0, Complexity: 33
3. Fitness: 713.68751, Fitness_val = 702.56003, G

 22%|██▎       | 135/600 [2:07:50<27:18:44, 211.45s/it]

Initial: 494, Deduplicated: 59, Operation simplified: 0, Constant: 58, Complex: 1, Final: 376
Population mean complexity: 40.651595744680854
Population mean fitness: 727.360185504531

0. Fitness: 713.79290, Fitness_val = 702.55540, Genome: ((([0.04945817] * (([1.36981057] + abs(x[1])) + [0.18322183])) - (([0.04945817] * (([0.04945817] * x[0]) - [-0.96954168])) - [-0.96954168])) + [10.49768355]), Age: 0, Complexity: 20
1. Fitness: 713.83646, Fitness_val = 702.55557, Genome: ((([0.04945817] * (abs(([0.04945817] * ([1.4561566] + abs(x[1])))) - [10.49768355])) - [0.40570983]) + [10.49768355]), Age: 0, Complexity: 15
2. Fitness: 713.78369, Fitness_val = 702.55940, Genome: (((([0.04945817] * ([1.36981057] + abs(([0.04945817] * ([1.36981057] + abs(([0.04945817] * x[1]))))))) * ([-0.88752803] - (([1.36981057] + abs(x[1])) + [0.18322183]))) - ([0.04945817] * (([1.36981057] + abs(x[1])) + [10.49768355]))) + [10.49768355]), Age: 0, Complexity: 33
3. Fitness: 713.78980, Fitness_val = 702.55942, Ge

 24%|██▍       | 147/600 [3:08:43<9:41:35, 77.03s/it]  


KeyboardInterrupt: 

In [17]:

problem = np.load("data/problem_8.npz")
x = problem["x"]
y = problem["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)
x_train, y_train, x_val, y_val = split_dataset(x,y,0.8)
print("x_train.shape: ", x_train.shape)
print("x_val.shape: ", x_val.shape)
print("y_train.shape: ", y_train.shape)
print("y_val.shape: ", y_val.shape)
my_pop_8 = evolve(x_train,y_train,x_val,y_val,conf)


x.shape: (6, 50000)
y.shape: (50000,)
x_train.shape:  (6, 40000)
x_val.shape:  (6, 10000)
y_train.shape:  (40000,)
y_val.shape:  (10000,)
Population is creating with a size:  100
Population has created.
Individual fitness values are assigning


MemoryError: Unable to allocate 11.9 GiB for an array with shape (40000, 40000) and data type float64